# Debugging Ambient L4 Workshop

![Architecture](./images/ambient-ztunnel-arch.png)

This workshop provides a comprehensive understanding of Ambient Mode's components, their interactions, and how to effectively monitor and troubleshoot the service mesh. By the end, you'll be able to confidently deploy and maintain an Ambient Mode installation in your environment.

## Links

1. Ambient Mesh overview - https://istio.io/latest/docs/ambient/
2. Ambient Mesh documentation - https://ambientmesh.io/docs
3. zTunnel - https://github.com/istio/istio/blob/master/architecture/ambient/ztunnel.md

## Tools

1. [Install](https://kind.sigs.k8s.io/docs/user/quick-start/#installation) `kind`
2. Install latest Istioctl (1.25+) - `curl -L https://istio.io/downloadIstio | sh -`

## Deploy a Kubernetes Cluster

You have a few options for setting up your environment:

- [kind cluster on a mac](../environments/single_cluster/mac/kind.ipynb)
- [gke](../environments/single_cluster/gcp/gke.ipynb)
- [aws](../environments/single_cluster/aws/aws.ipynb)

2. Verify the cluster is up and running and is ready for Istio installation with the following command.

In [ ]:
istioctl x precheck

## Install Solo Istio Ambient Mode

Istio Ambient Mode is a new architecture and approach to service mesh that eliminates the need for sidecar proxies while maintaining the core benefits of service mesh functionality. The installation process involves several key components: the Istio control plane (istiod), the CNI plugin for pod networking, and zTunnel for secure L4 communication. This installation will set up an Ambient Mode environment that enables zero-trust security and observability without the overhead of traditional sidecar-based deployments.

Ambient Components:
* **Istio Control Plane (istiod)**: Manages the service mesh configuration, handles service discovery, and coordinates the overall mesh behavior. It's responsible for distributing configuration to other components and managing the mesh's control plane functionality.
* **CNI Plugin**: Handles pod networking and ensures proper network configuration for pods in the mesh. It's responsible for setting up the necessary network interfaces and routing rules to enable ambient mode networking.
* [**zTunnel**](https://github.com/solo-io/ztunnel/blob/build/release-1.25/README.md): Provides secure L4 (transport layer) communication between services. It handles mTLS encryption, authentication, and secure tunneling of traffic between services, ensuring zero-trust security at the infrastructure level.
* **Waypoint Proxy** (optional): Provides L7 (application layer) capabilities when needed, such as advanced traffic management, observability, and security features. It can be deployed on-demand for services that require these additional capabilities.

### Install Istio Control Plane

The Istio Control Plane installation consists of two main components:

1. **Istio Base**: The foundation layer that installs the necessary Custom Resource Definitions (CRDs) and other base components required for the service mesh to function. This includes the core Istio APIs and configuration resources.
2. **Istiod**: The control plane component that manages the service mesh. It's responsible for:
  * Service discovery and configuration distribution
  * Certificate management and mTLS
  * Traffic management policies
  * Security policies
  * Mesh configuration

The Solo.io build of Istio includes several enhancements over the upstream version:

* **Enhanced L7 Telemetry**: Built-in support for detailed application-layer observability, including:
  * HTTP/gRPC request/response metrics
  * Detailed access logs
  * Distributed tracing integration
  * Custom metrics collection

* **Enterprise Features**: Additional capabilities like:
  * Advanced security features
  * Improved multi-cluster support
  * Enhanced debugging tools
  * Production-grade support

In [ ]:
# Set Istio Version
export ISTIO_VER=1.25.2
export ISTIO_HELM_CHART=oci://us-docker.pkg.dev/soloio-img/istio-helm
export ISTIO_REPO=us-docker.pkg.dev/soloio-img/istio

# Install Istio Base
helm upgrade -i istio-base "$ISTIO_HELM_CHART"/base                           \
    --version "${ISTIO_VER}-solo"                                             \
    --namespace istio-system                                                  \
    --create-namespace                                                        \
    --wait

# Install Istio Control Plane
helm upgrade -i istiod "$ISTIO_HELM_CHART"/istiod                             \
    --version "${ISTIO_VER}-solo"                                             \
    --namespace istio-system                                                  \
    --set profile=ambient                                                     \
    --set license.value=$GLOO_MESH_LICENSE_KEY                                \
    --set "hub=${ISTIO_REPO}"                                                 \
    --set "tag=${ISTIO_VER}-solo"                                             \
    --wait

### Install Istio Dataplane

![zTunnel Architecture](./images/ztunnel-architecture.png)

The Istio Dataplane installation in Ambient Mode consists of two critical components that work together to provide secure, efficient service-to-service communication.

1. **Istio CNI Plugin**:
  * Handles pod networking configuration at the node level
  * Sets up the necessary network interfaces and routing rules
  * Enables transparent traffic interception for Ambient Mode
  * Configures DNS capture for service discovery
  * Manages network policies and security rules
2. **zTunnel**:
  * Provides secure L4 (transport layer) communication
  * Implements zero-trust security principles
  * Handles mTLS encryption and authentication
  * Manages secure tunneling between services
  * Provides connection-level observability

Key Features:
* **Zero-Trust Security**: Built-in mTLS encryption for all service-to-service communication
* **Efficient Resource Usage**: No sidecar proxies required, reducing resource overhead
* **Transparent Operation**: Services communicate without awareness of the mesh
* **L4 Observability**: Connection-level metrics and logging
* **DNS Integration**: Automatic service discovery through DNS capture

> You can ignore the following warning `deprecated since v1.30; use the "appArmorProfile" field instead` as it will be fixed in later versions. 

In [ ]:
# Install Istio CNI Plugin
helm upgrade -i istio-cni "$ISTIO_HELM_CHART"/cni                             \
    --version "${ISTIO_VER}-solo"                                             \
    --namespace istio-system                                                  \
    --set profile=ambient                                                     \
    --set "hub=${ISTIO_REPO}"                                                 \
    --set "tag=${ISTIO_VER}-solo"                                             \
    --set "ambient.dnsCapture=true"                                           \
    --wait

# Install zTunnel
helm upgrade -i ztunnel "$ISTIO_HELM_CHART"/ztunnel                           \
    --version "${ISTIO_VER}-solo"                                             \
    --namespace istio-system                                                  \
    --set "hub=${ISTIO_REPO}"                                                 \
    --set "tag=${ISTIO_VER}-solo"                                             \
    --set profile=ambient                                                     \
    --wait

### Install Observability Tools

In this section, we'll install the standard Istio observability stack to help visualize and monitor the service mesh. This includes:
* **Prometheus**: For metrics collection and storage
* **Grafana**: For metrics visualization and dashboards
* **Kiali**: For service mesh visualization and management
* **Metrics Server**: For Kubernetes metrics collection

> Note: The observability stack installed here is the default development configuration provided by Istio. While these tools are excellent for development, testing, and debugging, they are not configured for production use. In a production environment, you would want to:
> * Configure proper resource limits and requests
> * Set up persistent storage
> * Implement proper security controls
> * Configure high availability
> * Use production-grade monitoring solutions

For production deployments, consider using:
* Enterprise monitoring solutions
* Managed observability services
* Custom-configured Prometheus with proper scaling
* Production-grade Grafana with proper authentication and authorization


Components:
* Grafana - https://istio.io/latest/docs/ops/integrations/grafana/ 
* Kiali - https://istio.io/latest/docs/ops/integrations/kiali/
* Prometheus - https://istio.io/latest/docs/ops/integrations/prometheus/

In [ ]:
kubectl apply -f data/metrics-server.yaml
kubectl apply -f https://raw.githubusercontent.com/istio/istio/release-1.25/samples/addons/prometheus.yaml
kubectl apply -f https://raw.githubusercontent.com/istio/istio/release-1.25/samples/addons/kiali.yaml
kubectl apply -f https://raw.githubusercontent.com/istio/istio/release-1.25/samples/addons/grafana.yaml

### Verify Installation

Verify that the monitoring components were installed correctly.

In [ ]:
kubectl get pods -n istio-system

## Install Applications

We are going to install a client and server application to communicate with each other. These applications will deploy to separate nodes so that we can show the communication path utilizing multiple zTunnel Proxies. The test applications will consist of a client running on one node which makes calls to a server running on another node. A traffic generator has also been deployed to constantly trigger the client to make calls to the server. 

We will deploy the applications first without service mesh and test their connectivity. In the next exercise we will add zTunnel interception and observe.

In [ ]:
# Deploy applications
kubectl create namespace client
kubectl create namespace server
kubectl apply -f data/client.yaml -n client --wait
kubectl apply -f data/server.yaml -n server --wait
kubectl apply -f data/traffic-gen-pod.yaml

### Verify communication

Let's connect to the client pod and start to make requests to the server to verify connectivity.

In [ ]:
kubectl exec -it -n client -c client $(kubectl get pod -n client -l app=client -o jsonpath='{.items[0].metadata.name}') -- sh -c "curl -s http://server.server.svc.cluster.local:8080/hello"

### Enable Ambient on client and server namespace

![zTunnel](./images/ztunnel.png)

Enabling Ambient Mode on a namespace fundamentally changes how traffic flows between services. Here's what happens when we add the `istio.io/dataplane-mode=ambient` label to a namespace:

1. **Before Ambient Mode**:
  * Services communicate directly using standard Kubernetes networking
  * No service mesh features are available
  * Traffic flows directly between pods without any interception
  * No mTLS, observability, or traffic management capabilities
2. **After Enabling Ambient Mode**:
  * The CNI plugin automatically intercepts all pod traffic
  * Traffic is redirected through the zTunnel proxy on each node
  * Services gain immediate access to:
  * mTLS encryption
  * L4 observability
  * Basic traffic management
  * No pod restarts or sidecar injection required

3. **Request Flow Changes**: 
```txt
Before:
  Client Pod → Direct Network → Server Pod

After:
  Client Pod → Node zTunnel → Network → Node zTunnel → Server Pod
```

4. **Key Benefits**:
  * Zero-trust security is automatically enabled
  * Traffic is automatically encrypted with mTLS
  * Connection-level metrics are collected
  * No application changes required
  * No pod restarts needed

The transition to Ambient Mode is transparent to the applications, requiring no changes to the services themselves. This makes it an ideal way to incrementally adopt service mesh capabilities without disrupting existing workloads.

Let's enable both the client and servers traffic to be captured by Istio's Ambient mode. 

In [ ]:
kubectl label namespace client istio.io/dataplane-mode=ambient
kubectl label namespace server istio.io/dataplane-mode=ambient

### Verify Traffic is now on Ambient

To verify that traffic is being properly intercepted and handled by Ambient Mode, we can examine the zTunnel logs on the nodes where our client and server pods are running. The zTunnel logs provide detailed information about the traffic flow, including:

1. **Connection Details**:
  * Source and destination pod information
  * Service identities (mTLS certificates)
  * Protocol and port information
  * Request/response status
2. **Traffic Flow Verification**:
  * Look for http access log entries showing successful connections
  * Verify the presence of service identities in the logs
  * Confirm traffic is being routed through zTunnel
  * Check for mTLS encryption indicators
3. **Key Log Indicators**:
  * `direction="outbound"` or `direction="inbound"` showing traffic direction
  * `src.identity` and `dst.identity` showing mTLS authentication
  * `protocol=HTTP1` indicating protocol type
  * `response_code=200` showing successful connections
4. **Expected Log Pattern**:
```text
   [timestamp] info http access request complete
   src.addr=[source-ip]:[port]
   src.workload="[source-pod]"
   src.namespace="[source-namespace]"
   src.identity="[source-identity]"
   dst.addr=[dest-ip]:[port]
   dst.service="[dest-service]"
   dst.workload="[dest-pod]"
   dst.namespace="[dest-namespace]"
   dst.identity="[dest-identity]"
   direction="[inbound/outbound]"
   method=[HTTP-method]
   path=[request-path]
   protocol=[protocol]
   response_code=[status-code]
```

In [ ]:
## Verify Ambient is enabled
printf "Checking zTunnel On Node ambient-worker\n"
zTunnelClientNode=$(kubectl get pods -n istio-system --field-selector spec.nodeName=ambient-worker -l app=ztunnel -o name)
kubectl logs -n istio-system $zTunnelClientNode | grep "http access" | tail -n 3
printf "\n\n"
printf "Checking zTunnel On Node ambient-worker2\n"
zTunnelServerNode=$(kubectl get pods -n istio-system --field-selector spec.nodeName=ambient-worker2 -l app=ztunnel -o name)
kubectl logs -n istio-system $zTunnelServerNode | grep "http access" | tail -n 3

## Observability

![kiali](./images/kiali.png)

Now that we have Ambient Mode enabled and traffic flowing through the mesh, we can visualize and analyze the service mesh behavior using the observability tools we installed earlier. This section focuses on using Kiali and Grafana to gain insights into the mesh's operation.

**Kiali Dashboard**

Kiali provides a comprehensive visualization of the service mesh, showing:
* Service-to-service communication patterns
* Traffic flow between services
* mTLS encryption status
* Request rates and response codes
* Error rates and latency metrics

To access the Kiali dashboard:

In [ ]:
istioctl dashboard kiali

### Verifying Ambient Traffic in Kiali
When examining the Kiali dashboard, there are several key indicators that confirm Ambient Mode is working correctly:
1. Service Graph View:
  * Look for the client and server services connected by a line
  * The line should be solid (not dashed) indicating active traffic
  * Hover over the connection to see:
    * Request rates
    * Error rates
    * Response times
    * Protocol information
2. mTLS Status:
  * Check the security badge on the service graph
  * Should show "mTLS: Enabled" for both services
  * This confirms that traffic is being encrypted by zTunnel
3. Traffic Metrics:
  * In the service details view, verify:
    * Inbound and outbound traffic is present
    * Request rates match your test traffic
    * Response codes show successful requests (200s)
    * No unexpected errors or failures
4. Workload View:
  * Confirm that workloads are shown without sidecars
  * Look for the "Ambient" label on the workloads
  * Verify that traffic is being routed through zTunnel
5. Traffic Distribution:
  * Check that traffic is flowing between the correct services
  * Verify that the traffic paths match the expected Ambient Mode routing
  * Confirm that no direct pod-to-pod traffic is occurring
6. Health Indicators:
  * Services should show as "Healthy"
  * No critical errors or warnings should be present
  * Traffic should be flowing without interruption

These indicators help confirm that:
  * Ambient Mode is properly enabled
  * Traffic is being correctly intercepted
  * mTLS is working as expected
  * Services are communicating securely
  * The mesh is functioning as designed

### Grafana Monitoring

![Grafana](./images/zTunnel-grafana.png)

Istio provides a comprehensive set of pre-built Grafana dashboards that help monitor and analyze the health of your service mesh deployment. These dashboards are automatically installed with the observability stack and provide insights at different levels of the mesh:

1. Istio Service Dashboard:
  * Service-level metrics and health indicators
  * Request rates and latencies
  * Error rates and types
  * Protocol-specific metrics
  * Client and server performance
2. Istio Workload Dashboard:
  * Workload-specific performance metrics
  * Resource utilization
  * Request success rates
  * Response time distributions
  * Error breakdowns
3. Istio Control Plane Dashboard:
  * istiod performance metrics
  * Configuration distribution status
  * Resource usage of control plane components
  * Error rates and types
  * Cache hit/miss ratios
4. Istio Mesh Dashboard:
  * Mesh-wide overview
  * Global request rates
  * Error distribution
  * Traffic patterns
  * Security metrics
5. Istio Ztunnel Dashboard
  * Versions deployed
  * Resource consumption
  * Traffic flow
  * DNS requests
  * xDS communication

Run the following command to explore the Grafana dashboards. 

In [ ]:
istioctl dashboard grafana

### Navigate to Dashboards
* Click on the "Dashboards" icon in the left sidebar (looks like four squares)
  * Select "Browse" from the dropdown menu
* You'll see several pre-configured Istio dashboards:
  * Istio Service Dashboard
  * Istio Workload Dashboard
  * Istio Control Plane Dashboard
  * Istio Mesh Dashboard
  * Istio Ztunnel Dashboard

## Troubleshooting 

In this section, we'll explore how to diagnose and resolve common issues in an Ambient Mode deployment. We'll use the tools and knowledge gained from previous sections to identify and fix problems.

**Common Issues and Diagnostic Steps**
1. Network Policy Issues:
  * Apply a restrictive NetworkPolicy to simulate connectivity problems
  * Use zTunnel logs to identify connection failures
  * Check Kiali for traffic flow disruptions
  * Verify mTLS status in the service graph
2. zTunnel Logging:
  * Adjust log levels to focus on specific issues
  * Monitor connection attempts and failures
  * Check for authentication and encryption problems
  * Verify traffic interception
3. Metrics and Monitoring:
  * Use Grafana dashboards to identify:
    * Increased error rates
    * Latency spikes
    * Connection failures
    * Resource constraints
  * Check Kiali for:
    * Broken service connections
    * Failed mTLS handshakes
    * Traffic routing issues
4. Control Plane Health:
  * Monitor istiod logs for configuration issues
  * Check control plane metrics for:
    * Configuration distribution problems
    * Resource utilization
    * Error rates
  * Verify certificate management
5. Diagnostic Tools:
  * Use istioctl commands for:
    * Configuration validation
    * Proxy status checks
    * Certificate verification
  * Check pod logs for application-level issues
  * Verify network policies and service configurations

This section will help you develop a systematic approach to troubleshooting Ambient Mode deployments, using the observability tools and logs to identify and resolve issues.


First lets introduce a network policy that will deny all traffic to the `server`. This will serve as our traffic issue we will need to investigate.

In [ ]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: deny-all-to-server
  namespace: server
spec:
  podSelector:
    matchLabels:
      app: server
  policyTypes:
    - Ingress
  ingress: []
EOF

At scale it may not be advisable to print every access log that travels through zTunnel. We are going to update zTunnel to only log errors.

In [ ]:
# Install zTunnel with log level set to error
helm upgrade -i ztunnel "$ISTIO_HELM_CHART"/ztunnel                           \
    --version "${ISTIO_VER}-solo"                                             \
    --namespace istio-system                                                  \
    --set "hub=${ISTIO_REPO}"                                                 \
    --set "tag=${ISTIO_VER}-solo"                                             \
    --set profile=ambient                                                     \
    --set "l7Telemetry.accessLog.enabled=false"                               \
    --set "l7Telemetry.accessLog.skipConnectionLog=true"                      \
    --set "l7Telemetry.distributedTracing.enabled=false"                      \
    --set logLevel="info\,access=warn"                                        \
    --wait

## Verify Traffic

Let's go ahead and see if traffic from the client is able to reach the server.

In [ ]:
kubectl exec -it -n client -c client $(kubectl get pod -n client -l app=client -o jsonpath='{.items[0].metadata.name}') -- sh -c "curl -s http://server.server.svc.cluster.local:8080/hello"

* Look at access logs on ztunnel where server is running

In [ ]:
printf "Checking zTunnel On Node ambient-worker2\n"
zTunnelServerNode=$(kubectl get pods -n istio-system --field-selector spec.nodeName=ambient-worker -l app=ztunnel -o name)
kubectl logs -n istio-system $zTunnelServerNode | grep "connection complete" | tail -n 3

## Teardown

In [ ]:
kind delete cluster --name ambient